In [1]:
import sys
import json
import cobra
import cplex
import re
import cobrakbase
import escher

from escher import Builder
from optlang.symbolics import Zero, add
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np

from multiprocessing import Process
import time

cobrakbase 0.2.7


In [3]:
#input coeficient values to cycle
res=[1e-8,1e-6,1e-4,1e-2,1,100]
#latestart
res2=[1e-8,1e-6,1e-4,1e-2,1,100]
#res2=[1,100,1e4]
tissues = ["Mature_leaf"]

In [ ]:
############Coefficient for kcat fitting:
kcat_fit = 0
###########Coefficient for flux fitting:
#flux_fit = 1
############Coefficient for kinetic fitting:
kinetic_fit = 0
results=[]
for x in res2:
    #kinetic_fit=x
    #kcat_fit=x
    flux_fit=x
    for x in res:
        #flux_fit=x
        #kcat_fit=x
        kinetic_fit=x
        for x in res:
            #kinetic_fit=x
            kcat_fit=x
            #flux_fit=x
            
            if flux_fit == 0.0001 and kinetic_fit == 0.0001 and kcat_fit==100:
                continue
#             elif flux_fit == 0.0001 and kinetic_fit == 0.0001 and kcat_fit==10000:
#                 continue
#             elif flux_fit == 0.01 and kinetic_fit == 0.01 and kcat_fit==10000:
#                 continue
            elif flux_fit == 1 and kinetic_fit == 0.0001 and kcat_fit==100:
                continue
                
                

            #####################reset base model
        #     base_model=None
        #     base_model = clone_model
            model =cobra.io.read_sbml_model('C4_HS_Zmays_Model_v2.xml')
            model.solver = 'optlang-cplex'
            #Setting model to complete media
            #media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
            pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
            pkgmgr.getpkg("KBaseMediaPkg",1).build_package(None)

            #Reading reaction protein abundances
            data = ""
            with open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/ReactionProtein.txt', 'r') as file:
                data = file.read()
            lines = data.split("\n")
            reaction_proteins = {}
            headers = None
            for line in lines:
                if headers == None:
                    headers = line.split("\t")
                else:
                    array = line.split("\t")
                    for i in range(2, len(array)):
                        if headers[i] not in reaction_proteins:
                            reaction_proteins[headers[i]] = dict()
                        reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

            #Reading reaction kcat values
            data = ""
            with open('Kcats_grouped.txt', 'r') as file:
                data = file.read()
            lines = data.split("\n")
            reaction_kcat = {}
            headers = None
            for line in lines:
                if headers == None:
                    headers = line.split("\t")
                else:
                    array = line.split("\t")
                    reaction_kcat[array[0]] = abs(float(array[1]))

            #Reading measured reaction fluxes
            data = ""
            with open('measured_Claudia4_edit.txt', 'r') as file:
                data = file.read()
            lines = data.split("\n")
            reaction_measures = {}
            headers = None
            for line in lines:
                if headers == None:
                    headers = line.split("\t")
                else:
                    array = line.split("\t")
                    if array[1] not in reaction_measures:
                            reaction_measures[array[1]] = dict()
                    rxnid = array[0]
                    reaction_measures[array[1]][rxnid] = abs(float(array[2]))

            biomass_fluxes = {
                "root_MZ" : 0.049925,
                "root_EZ" : 0.049925,
                "root_Cortex" : 0.049925,
                "root_Stele" : 0.049925,
                "Embryo_20_DAP" : 0.015591667,
                "Embryo_38_DAP" : 0.015591667,
                "Endosperm_8_DAP" : 0.010208333,
                "Endosperm_10_DAP" : 0.010208333,
                "Endosperm_12_DAP" : 0.010208333,
                "Endosperm_Crown_27_DAP" : 0.010208333,
                "Pericarp_Aleurone_27_DAP" : 0.001529167,
                "GermEmbryo_2_DAI" : 0.015591667,
                "Zone_1" : 0.006395833,
                "Zone_2" : 0.006395833,
                "Zone_3" : 0.006395833,
                "Mature_leaf" : 0.006395833
            }

            protein_fraction = {
                'root_MZ': 0.05,
                'root_EZ' :0.05,
                'root_Cortex': 0.05,
                'root_Stele' : 0.05,
                'Embryo_20_DAP' : 0.116,
                'Embryo_38_DAP' : 0.116,
                'Endosperm_8_DAP' : 0.08,
                'Endosperm_10_DAP' : 0.08,
                'Endosperm_12_DAP' : 0.08,
                'Endosperm_Crown_27_DAP' : 0.08,
                'Pericarp_Aleurone_27_DAP' : 0.09,
                'GermEmbryo_2_DAI' : 0.116,
                'Zone_1': 0.0948,
                'Zone_2': 0.0948,
                'Zone_3': 0.0948,
                'Mature_leaf': 0.0948
            }

            rxn = model.reactions.get_by_id("bio1_biomass")
            rxn.upper_bound = 0.006395833
            rxn.lower_bound = 0.006395833
            sol=model.optimize()
            # builder.model = model
            # builder.reaction_data = sol.fluxes

            modellist = []
            count = 0
            base_model = None
            for tissue in tissues:
                #print("Tissue:"+tissue)
                #Safe cloning method
                clone_model = cobra.io.json.from_json(cobra.io.json.to_json(model))
                #Applying flexible biomass constraints
                pkgmgr = MSPackageManager.get_pkg_mgr(clone_model,1)
                protein = protein_fraction[tissue]
                protein_flux = -1*(0.2-protein)
                pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1_biomass","use_rna_class":[-0.75,0.75],
                    "use_dna_class":[-0.75,0.75],
                    "use_protein_class":[protein_flux,protein_flux],
                    "use_energy_class":[-0.1,0.1]})
                #Adding tissue specific proteome constraints
                package_params = {
                    "rxn_proteome":reaction_proteins[tissue],
                    #"target_flux":reaction_measures[tissue],
                    "flux_values":reaction_measures[tissue],
                    "prot_coef" : protein_fraction[tissue],#Set to the fraction of the cell that is protein
                    "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
                    "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
                    "set_objective":0
                }
                #Only base model fits kcat values
                if count == 0:
                    package_params["kcat_values"] = reaction_kcat
                pkgmgr.getpkg("ProteomeFittingPkg",1).build_package(package_params)
                #Constraining biomass flux to reference value
                rxn = clone_model.reactions.get_by_id("bio1_biomass")
                rxn.upper_bound = biomass_fluxes[tissue]
                rxn.lower_bound = biomass_fluxes[tissue]
                if count == 0:
                    base_model = clone_model
                else:
                    modellist.append(clone_model)
                count += 1




            #Pulling based model package manager
            pkgmgr = MSPackageManager.get_pkg_mgr(base_model,1)
            #Merging all models together
            if count > 1:
                pkgmgr.getpkg("ProblemReplicationPkg",1).build_package({"models":modellist,"shared_variable_packages":{pkgmgr.getpkg("ProteomeFittingPkg"):["kapp"]}})
            #Creating combined objective function
            objvars = []



            #Adding base model kfit,kvfit,and vfit to objective
            for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
                objvars.append(flux_fit * pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid] ** 2)
            for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"]: 
                objvars.append(kinetic_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid] ** 2)
            for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"]: 

                objvars.append(kcat_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid] ** 2)

            #Adding sub model kvfit,and vfit to objective
            count = 0
            for submodel in modellist:
                submgr = MSPackageManager.get_pkg_mgr(submodel)
                for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:

                    varname = submgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].name
                    objvars.append(kinetic_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
                for rxnid in submgr.getpkg("FluxFittingPkg").variables["vfit"]:
                    varname = submgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].name

                    objvars.append(flux_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
                count += 1
            #Setting the objective
            base_model.objective = base_model.problem.Objective(add(objvars), direction="min", sloppy=True)


            with open('ProteomeDrivenModeling.lp', 'w') as out:
                out.write(str(base_model.solver))
            


            try:
                sol=base_model.optimize()
            except:
                print('error')
                continue
               
 
            
            

            # buildertwo.model = base_model
            # buildertwo.reaction_data = sol.fluxes
            #print ('done solving')

            #Pulling solution data
            solution_data = {}
            count = 0
            for tissue in tissues:
                solution_data[tissue] = {"flux":{},"vfit":{},"kfit":{},"kvfit":{},"kapp":{},"flex":{}}     
                if count == 0:
                    for reaction in base_model.reactions:
                        if re.search('FLEX_(cpd\d+_c0)', reaction.id) != None:
                            m = re.search('FLEX_(cpd\d+_c0)', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = reaction.flux
                        elif re.search('([a-z]+)_flex', reaction.id) != None:
                            m = re.search('([a-z]+)_flex', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = reaction.flux
                        else:
                            solution_data[tissue]["flux"][reaction.id] = reaction.flux
                    for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
                        solution_data[tissue]["vfit"][rxnid] = pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"]:
                        solution_data[tissue]["kfit"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kapp"]:

                        solution_data[tissue]["kapp"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kapp"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:
                        solution_data[tissue]["kvfit"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].primal
                else:
                    submodel = modellist[count-1]
                    submgr = MSPackageManager.get_pkg_mgr(submodel)
                    for reaction in base_model.reactions:
                        if re.search('FLEX_(cpd\d+_c0)', reaction.id) != None:
                            m = re.search('FLEX_(cpd\d+_c0)', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flex"][m[1]] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                        elif re.search('([a-z]+)_flex', reaction.id) != None:
                            m = re.search('([a-z]+)_flex', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flex"][m[1]] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                        else:
                            solution_data[tissue]["flux"][reaction.id] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flux"][reaction.id] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                    for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kfit"]:
                        varname = submgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid].name
                        solution_data[tissue]["kfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal
                    for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:
                        varname = submgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].name
                        solution_data[tissue]["kvfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal
                    for rxnid in submgr.getpkg("FluxFittingPkg").variables["vfit"]:
                        varname = submgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].name
                        solution_data[tissue]["vfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal    
                count += 1


            ######## r2
            y_pred=[]
            y_true=[]
            for rxn in reaction_measures['Mature_leaf'].keys():
                if rxn in model.reactions:
                    y_true.append(reaction_measures['Mature_leaf'][rxn])
                    y_pred.append(abs(solution_data['Mature_leaf']['flux'][rxn]))
            r2=r2_score(y_true,y_pred)
            #results.append([kcat_fit,flux_fit,kinetic_fit,r2])
            print (kcat_fit,flux_fit,kinetic_fit,r2)
            #print (flux_fit,r2)


                

1e-08 1 1e-08 0.26778161047404814
1e-06 1 1e-08 0.26778162377141235
0.0001 1 1e-08 0.7474476250839768
0.01 1 1e-08 -0.08253330674877213
error
error
1e-08 1 1e-06 -0.02260354377187279
1e-06 1 1e-06 -0.0028232342942808852
0.0001 1 1e-06 -0.022374109402280018
0.01 1 1e-06 -0.04566868865197016
1 1 1e-06 -0.11030197313299017
error
1e-08 1 0.0001 -0.052501527175920826
1e-06 1 0.0001 -0.050058582195968704
0.0001 1 0.0001 -0.059822777375225344
0.01 1 0.0001 -0.07541547931974968
1 1 0.0001 -0.07874294655333913
1e-08 1 0.01 -0.07410454181749548
1e-06 1 0.01 -0.06976922197643876
0.0001 1 0.01 -0.05347565852116465
0.01 1 0.01 -0.06113956429816625
1 1 0.01 -0.07296864204120324
100 1 0.01 -0.058944456333234196
1e-08 1 1 -0.04847098695692975
1e-06 1 1 -0.06693222099537177
0.0001 1 1 -0.058772337652059914
0.01 1 1 -0.06742771205391618
1 1 1 -0.07224572652345884
100 1 1 -0.07225495873110299
1e-08 1 100 -1.948030458294887
1e-06 1 100 -2.3316197323921393
0.0001 1 100 -2.772292304939366
0.01 1 100 -2.9432

In [ ]:
df=pd.DataFrame(results)
#df.to_csv("coef_opt.csv")

In [6]:
df.nlargest(10,3)

KeyError: 3